In [2]:
%run Structural_Model.ipynb
class Environment():
    def __init__(self, aero_model, ref_ground_wind, latitude=32.9895472, longitude=-106.9694681, height=launch_site_alt):
        self.ka        = 1.4                   # Ratio of specific heats, air  
        self.Ra        = 287.058                 # Avg. specific gas constant (dry air)
        self.longitude = np.radians(longitude)
        self.latitude  = np.radians(latitude) # degrees north, launch site
        self.launch_pt = np.array([latitude, longitude])
        self.up = np.array([np.cos(self.longitude) * np.cos(self.latitude),
                            np.sin(self.longitude) * np.cos(self.latitude),
                            np.sin(self.latitude)])
        self.project_normal = np.identity(3) - np.outer(np.array([0,0,1]), np.array([0,0,1]))
        
        self.date_time = datetime(2021, 12, 10, 9, 0, 0)
        
        self.a = 6378137.0 # m, semimajor axis, defining parameter of wgs 84
        self.flattening_inv = 298.257223563 # 1/f, defining parameter along with semimajor axis
        self.mu_earth  = 3.986004418 * 10**14 # m^3/s^2, earth gravitational parameter
        
        # derived parameters
        self.f = 1 / self.flattening_inv
        self.b = 6356752.3142 # m, semiminor axis
        self.e = 0.081819190842622 # eccentricity approximation
        self.earth_rad = 6.378137e6 # m, earth mean equitorial radius
        self.earth_mass = 5.9733328e24 # kg, mass of earth including atmosphere
        
        # more derived parameters
        self.e2 = self.f * (2 - self.f)
        self.a1 = self.a * self.e2
        self.a2 = self.a1 * self.a1
        self.a3 = self.a1 * self.e2 * 0.5
        self.a4 = 2.5 * self.a2
        self.a5 = self.a1 + self.a3
        self.a6 = 1 - self.e2
        
        # ref markely & crassidis
        self.J2 = 1.08262668355e-3
        self.J3 = -2.53265648533e-6
        self.J4 = -1.61962159137e-6
        
        self.initial_position    = self.geodetic_to_ECEF(self.latitude, self.longitude, height)
        self.initial_height      = np.linalg.norm(self.initial_position)
        # this is kind of a hack, but it works
        self.initial_orientation = eulerangle_to_quat(self.longitude, np.pi/2 - self.latitude, 0)
        
        
        # International Gravity Formula (IGF) 1980, Geodetic Reference System 1980 (GRS80)
        self.IGF    = 9.780327 * (1 + 0.0053024 * np.sin(self.latitude)**2 - 0.0000058 * np.sin(2*self.latitude)**2)
        # Free Air Correction (FAC)
        self.FAC    = -3.086 * 10**(-6)
        
        # for coriolis accel
        self.sinlat = np.sin(self.latitude)
        self.coslat = np.cos(self.latitude)
        self.erot   = 7.2921150e-5 # Sidearial Earth rotation rate
        
        #openrocket method for gravitational accel, open rocket uses 6371000 m for earth radius
        # comparing is low priority
        #sin2lat = self.sinlat**2
        #self.g_0 = 9.7803267714 * ((1.0 + 0.00193185138639 * sin2lat) / np.sqrt(1.0 - 0.00669437999013 * sin2lat))
        
        self.aero_model = aero_model
        
        # initialize atmospheric model, is it worth calculating this once and for all?
        T_0 = 288.15 # K
        p_0 = 101325 # Pa
        self.layer =  [0, 11000, 20000, 32000, 47000, 51000, 71000, 84852,
                       100e3, 120e3, 140e3, 160e3, 180e3, 200e3]
                       #90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000]
        self.baseTemp = [288.15, 216.65, 216.65, 228.65, 270.65, 270.65, 214.65, 186.95,
                         1.88e2, 3.65e2, 6.1e2, 7.59e2, 8.53e2, 9.11e2]
                         #196.86, 203.06, 224.28, 250.09, 285.13, 364.19, 441.79, 517.94, 600]
        self.basePress = [p_0]
        for i, alt in enumerate(self.layer):
            if i == 0:
                pass
            else:
                self.basePress.append(self.getConditions(self.layer[i] - 1)[1])
        
        # references for wind model < 150 m
        self.u_0   = ref_ground_wind
        self.z_0   = 18.3
        self.alpha = 1.21 * self.u_0**(-0.75)
            
    # pushes a 2D vector into a 3D vector, for wind model
    def pushforward(self, lat, lon):
        ca = np.cos(lat)
        sa = np.sin(lat)
        co = np.cos(lon)
        so = np.sin(lon)
        
        d_phi = np.array([[-sa*co, -ca*so],
                          [-sa*so, ca*co],
                          [ca, 0]])
        return d_phi
        
    # https://github.com/openrocket/openrocket/blob/b5cde10824440a1e125067b7d149873deec424b4/core/src/net/sf/openrocket/util/GeodeticComputationStrategy.java
    # double check, low priority. might not even need
    def coriolis(self, v):
        v_e = v[0] # note, pretty sure openrocket does this wrong by adding a negative sign here
        v_n = v[1]
        v_u = v[2]
        #acc = -2*self.erot*np.cross(np.array([0, self.coslat, self.sinlat]), v)
        acc = -2*self.erot*np.cross(np.array([0, 0, 1]), v)
        return acc # acceleration
    
    # https://www.sensorsone.com/local-gravity-calculator/
    # maybe one day i'll use this alternative https://github.com/openrocket/openrocket/blob/unstable/core/src/net/sf/openrocket/models/gravity/WGSGravityModel.java
    def old_g_accel(self, x):
        return np.array([0, 0, -self.IGF + self.FAC * x[-1]])
        #return self.g_0 * (6356766/(6356766 + x[-1]))**2
        
    # gravitational torque. inputs in inertial frame, outputs in body frame
    def T_gg(self, x, attitude_BI, J):
        r = np.linalg.norm(x)
        n = frame_rotation(attitude_BI, - x / r)
        return (3 * self.mu_earth / r**3) * np.cross(n, J.dot(n)) # kg * m^2 / s^2

    # U.S. 1976 Standard Atmosphere, at some point compare more rigorously to openrocket's method, low priority
    #def old_std_at(self, x):
    #    if x[2] < 11000:
    #        T = 15.04 - 0.00649*x[2]
    #        p = 101.29*((T + 273.1)/288.08)**5.256
    
    #    elif 11000 <= x[2] and x[2] <25000:
    #        T = -56.46
    #        p = 22.65*np.exp(1.73 - 0.000157*x[2])
    
    #    else:
    #        T = -131.21 + 0.00299*x[2]
    #        p = 2.488 * ((T + 273.1)/216.6)**(-11.388)
    
    #    p_a = p*1000                 # Ambient air pressure [Pa]
    #    rho = p/(0.2869*(T + 273.1)) # Ambient air density [kg/m^3]
    #    T_a = T + 273.1              # Ambient air temperature [K]
    #    return np.array([p_a, rho, T_a])
    
    # is there a more intelligent way to do this?
    # why are there two formulae?, med priority
    def getConditions(self, altitude):
        index = 0
        g = np.linalg.norm(self.old_g_accel([altitude]))
        #altitude = altitude * 6356766 / (altitude + 6356766) # geopotential altitude
        for i in range(len(self.layer)-1):
            if self.layer[i + 1] > altitude:
                break
            index += 1
        if index > 12: index = 12
        rate = (self.baseTemp[index+1] - self.baseTemp[index])/(self.layer[index+1] - self.layer[index])
        t = self.baseTemp[index] + (altitude - self.layer[index])* rate
        if altitude > 95000:
            Ra = R_univ / 25 # kind of a hack. see miller '57 when you care
        else:
            Ra = self.Ra
        
        if abs(rate) > 0.001:
            p = self.basePress[index]*(1 + (altitude-self.layer[index])*rate/self.baseTemp[index])**(-g/(rate*Ra))
        else:
            p = self.basePress[index]*(np.exp(-(altitude-self.layer[index])* g / (Ra * self.baseTemp[index])))
        return (t, p)
    
    def std_at(self, x):
        T_a, p_a = self.getConditions(self.ECEF_to_geodetic(x)[2])
        #T_a, p_a = self.getConditions(np.linalg.norm(x) - self.initial_height)
        # you know this is a poor method for checking conditions, med priority
        if self.Ra == 287.058 and x[-1] > 95000:
            self.Ra = R_univ / 25 # rough estimate when you care, see
            # Miller, L. E. (1957). “Molecular weight” of air at high altitudes, low priority

        rho = p_a/(self.Ra*T_a) # Ambient air density [kg/m^3]
        
        # https://www.cfd-online.com/Wiki/Sutherland%27s_law
        dyn_visc = 1.458e-6 * T_a**(3/2) / (T_a + 110.4)
        mu = dyn_visc / rho if rho != 0 else 50
        #mu = 3.7291*10**-6 + 4.9944 * T_a / rho*10**8 # kinematic viscosity, per openrocket
        return np.array([p_a, rho, T_a, mu])
    
    def atmo(self, x):
        f107a = 150
        f107 = 150
        ap = 4
        
        lat, long, h = self.ECEF_to_geodetic(x)
        h = h/1000
        
        time = self.date_time
        year = time.year
        doy = int(time.strftime("%j"))
        iyd = int(str(year)[-2:] + str(doy))
        sec = (time.hour * 3600.
                    + time.minute * 60.
                    + time.second
                    + time.microsecond * 1e-6)
        lst = sec / 3600. + long / 15.0
        
        densities, temps = nrlmsise00.msise_model(self.date_time, h, lat, long, f107a, f107, ap, lst)
        winds = pyhwm2014.hwm14.hwm14(iyd=iyd, sec=sec, alt=h, glat=lat, glon=long, stl=lst, f107a=f107a, f107=f107, ap=[-1, ap])
        rho = densities[5] * 1000 #  mass density, kg/m^3
        T_a = temps[1] # K, should be neutral temp (not exospheric)
        
        #winds = hwm93.run(time=self.date_time, altkm=h, glat=lat, glon=long, f107a=f107a, f107=f107, ap=ap)
        #atmos = msise00.run(time=self.date_time, altkm=h, glat=lat, glon=long, indices={'f107s':f107a, 'f107':f107, 'Ap':ap})
        #rho = atmos.Total.values[0][0][0][0] # mass density, kg/m^3
        #T_a = atmos.Tn.values[0][0][0][0] # K, should be neutral temp (not exospheric)
        
        p_a = rho * T_a * self.Ra
        dyn_visc = 1.458e-6 * T_a**(3/2) / (T_a + 110.4)
        mu = dyn_visc / rho
        
        
        d_phi = self.pushforward(lat, long)
        v_wind = d_phi.dot(np.array(winds))
        #v_wind = d_phi.dot(np.array([winds.meridional.values, winds.zonal.values])).flatten()
        
        return np.array([p_a, rho, T_a, mu]), v_wind
        
    
    # calculates drag force, etc
    def aero(self, state, rkt, air, wind):
        x, q, v, w = state[1:]
        roll = w[2]
        p_a, rho, T_a, mu = air
        
        # Check Knudsen number and consider other drag models (e.g. rarified gas dyn vs. quadratic drag), low priority
        v0 = np.linalg.norm(v - wind)
        v_hat = normalize(v - wind)
        v_body = frame_rotation(q, v_hat)
        
        alpha = np.arccos(np.clip(np.dot(v_body, np.array([0,0,1])), -1, 1))
        sound_speed = np.sqrt(self.ka * self.Ra * T_a)
        fin_flutter = self.aero_model.flutter_velocity(sound_speed, p_a) * 0.85 # 15% factor of safety
        Ma = v0 / sound_speed # Mach
        #Ma = v0 / (165.77 + 0.606*T_a) # openrocket's approx, < 1% error at low altitude, around 5-7% at high, low priority
        dyn_press = 0.5 * rho * v0**2         # Dynamic pressure [Pa]
        
        # HIGH priority, get parasitic drag from fuel piping :(
        # at some point include an option for the old look up table here, med priority
        # also med priority, but a simple planform calculation for CoP could be useful for low-fidelity model
        # low priority, consider "Active Control Stabilization of High Power Rocket" for drag equations
        if rkt.descending:
            CoP, CN, CDax, Cld_times_d, C_damp_p, C_damp_y = 0, 0, 0, 0, 0, 0
        else:
            CoP, CN, CDax, Cld_times_d, C_damp_p, C_damp_y = self.aero_model.physics(alpha, Ma, v0, w, mu, rkt.CoM[2])
        CoP = np.array([0,0, rkt.length - CoP]) # CoP calculated nose ref, but we work base ref
        
        direction = -normalize(self.project_normal.dot(v_body)) # normalize might be unnecessary
        norm_force= CN*direction
        
        if not rkt.off_tower:
            norm_force *= 0
            Cld_times_d = 0
            C_damp_p, C_damp_y = 0, 0
            
        pitch_moment = np.cross(CoP - rkt.CoM, norm_force)
        #C_damp_p = min(C_damp_p, pitch_moment[0])
        #C_damp_y = min(C_damp_y, pitch_moment[1])
        pitch_damp   = np.array([C_damp_p * np.sign(w[0]), C_damp_y * np.sign(w[1]), 0])
        #if rkt.descending:
            #print(alpha)
            #print('norm:',CN, '. damp:',C_damp_p, C_damp_y)
        mult = dyn_press * rkt.frontal_area
        
        torque_body = (np.array([0, 0, -Cld_times_d]) +
                       pitch_moment + pitch_damp #+
                       #np.array([np.random.uniform(-0.0005,0.0005),
                       #         np.random.uniform(-0.0005,0.0005), 0]) # openrocket injects noise for realism
                      ) * mult
        
        if rkt.descending:
            force_body = -mult * self.aero_model.tumbling_drag * v_hat
        else:
            force_body = (np.array([0, 0, -CDax]) + norm_force) * mult
            
        return np.array([force_body, torque_body, v0, dyn_press, Ma, alpha, CoP[2], fin_flutter, CN, CDax], dtype=object)
    
    # https://en.wikipedia.org/wiki/Wind_profile_power_law
    # assuming stable atmosphere over land without too rough
    # of terrain and wind measurements at 10 m
    def ground_wind_profile(self, z):
        if z < 0: z = 0
        if z > 150: z = 150
        return self.u_0 * (z / self.z_0)**self.alpha
    
    def wind_vector(self, x):
        magnitude = self.ground_wind_profile(x[2] - launch_site_alt)
        return np.array([1, 0, 0]) * magnitude
    
    # transformation from WGS-84 geodetic coordinates to ECEF geocentric coordinates
    def geodetic_to_ECEF(self, lat, long, h):
        N = self.a / np.sqrt(1 - (self.e*np.sin(lat))**2)
        temp = (N + h) * np.cos(lat)
        x = temp * np.cos(long)
        y = temp * np.sin(long)
        #z = (N*(1 - self.e**2) + h) * np.sin(lat) # from textbook
        z = (N*(self.b / self.a)**2 + h) * np.sin(lat) # from wgs def
        return np.array([x, y, z])

    # transformation from ECEF geocentric coordinates to WGS-84 geodetic coordinates
    def ECEF_to_geodetic_old(self, r):
        x = r[0]
        y = r[1]
        z = r[2]
        def geo_rho(x, y):
            return np.linalg.norm(np.array([x, y]))

        def geo_e_squared(a, b):
            return 1 - (b/a)**2

        def geo_eps_squared(a, b):
            return (a/b)**2 - 1

        def geo_p(z, eps2):
            return abs(z) / eps2

        def geo_s(rho, e2, eps2):
            return rho**2 / (e2*eps2)

        def geo_q(p, b, s):
            return p**2 - b**2 + s

        def geo_u(p, q):
            return p / np.sqrt(q)

        def geo_v(b, u, q):
            return (b*u)**2 / q

        def geo_P(v, s, q):
            return 27*v*s/q

        def geo_Q(P):
            return (np.sqrt(P+1) + np.sqrt(P))**(2/3)

        def geo_t(Q):
            return (1 + Q + 1/Q)/6

        def geo_c(u, t):
            return np.sqrt(u**2 - 1 + 2*t)

        def geo_w(c, u):
            return (c - u)/2

        def geo_d(z, q, u, v, w, t):
            return np.sign(z)*np.sqrt(q)*(w + np.sqrt(np.sqrt(t**2 + v) - u * w - t*0.5 - 0.25))

        def geo_N(a, eps2, d, b):
            return a * np.sqrt(1+ eps2*(d/b)**2)

        # latitude
        def geo_lam(eps2, d, N):
            return np.arcsin((eps2 + 1)*d/N)

        # height
        def geo_h(rho, z, a, N, lam): # height above geode
            return rho*np.cos(lam) + z*np.sin(lam) - a**2 / N

        # longitude
        def geo_phi(x, y):
            return np.arctan2(y, x)
        
        a = self.a
        b = self.b
        e2 = geo_e_squared(a, b)
        eps2 = geo_eps_squared(a, b)
        rho = geo_rho(x, y)
        p = geo_p(z, eps2)
        s = geo_s(rho, e2, eps2)
        q = geo_q(p, b, s)
        u = geo_u(p, q)
        v = geo_v(b, u, q)
        P = geo_P(v, s, q)
        Q = geo_Q(P)
        t = geo_t(Q)
        c = geo_c(u, t)
        w = geo_w(c, u)
        d = geo_d(z, q, u, v, w, t)
        N = geo_N(a, eps2, d, b)

        lam = geo_lam(eps2, d, N)
        h = geo_h(rho, z, a, N, lam)
        phi = geo_phi(x, y)
        return np.array([np.degrees(lam), np.degrees(phi), h]) # lat, long, height
    
    # refer to Markely and Crassidis
    def g_accel(self, position):
        r = np.linalg.norm(position)
        xoverr = position[0] / r
        yoverr = position[1] / r
        zoverr = position[2] / r
        zoverrsquare = zoverr**2
        zoverrcube = zoverr*zoverrsquare
        zoverr4th = zoverrsquare**2
        muoverrsquared = self.mu_earth / r**2
        rearthoverr = self.earth_rad / r
        a   = - position / r
        aj2 = -3/2 * self.J2 * rearthoverr**2 * np.array([(1 - 5 * zoverrsquare) * xoverr,
                                                          (1 - 5 * zoverrsquare) * yoverr,
                                                          (3 - 5 * zoverrsquare) * zoverr])
        aj3 = -1/2 * self.J3 * rearthoverr**3 * np.array([5 * (7 * zoverrcube - 3 * zoverr) * xoverr,
                                                          5 * (7 * zoverrcube - 3 * zoverr) * yoverr,
                                                          3 * (10 * zoverrsquare - 35/3 * zoverr4th - 1)])
        aj4 = -5/8 * self.J4 * rearthoverr**4 * np.array([(3 - 42 * zoverrsquare + 63 * zoverr4th) * xoverr,
                                                          (3 - 42 * zoverrsquare + 63 * zoverr4th) * yoverr,
                                                          -(15 - 70 * zoverrsquare + 63 * zoverr4th) * zoverr])
        return muoverrsquared * (a + aj2 + aj3 + aj4)
    
    # source: https://possiblywrong.wordpress.com/2014/02/14/when-approximate-is-better-than-exact/
    def ECEF_to_geodetic(self, ecef):
        """Convert ECEF (meters) to LLA (radians and meters).
        """
        # Olson, D. K., Converting Earth-Centered, Earth-Fixed Coordinates to
        # Geodetic Coordinates, IEEE Transactions on Aerospace and Electronic
        # Systems, 32 (1996) 473-476.
        w = math.sqrt(ecef[0] * ecef[0] + ecef[1] * ecef[1])
        z = ecef[2]
        zp = abs(z)
        w2 = w * w
        r2 = z * z + w2
        r  = math.sqrt(r2)
        s2 = z * z / r2
        c2 = w2 / r2
        u = self.a2 / r
        v = self.a3 - self.a4 / r
        if c2 > 0.3:
            s = (zp / r) * (1 + c2 * (self.a1 + u + s2 * v) / r)
            lat = math.asin(s)
            ss = s * s
            c = math.sqrt(1 - ss)
        else:
            c = (w / r) * (1 - s2 * (self.a5 - u - c2 * v) / r)
            lat = math.acos(c)
            ss = 1 - c * c
            s = math.sqrt(ss)
        g = 1 - self.e2 * ss
        rg = self.a / math.sqrt(g)
        rf = self.a6 * rg
        u = w - rg * c
        v = zp - rf * s
        f = c * u + s * v
        m = c * v - s * u
        p = m / (rf / g + f)
        lat = lat + p
        if z < 0:
            lat = -lat
        return (np.degrees(lat), np.degrees(math.atan2(ecef[1], ecef[0])), f + m * p / 2)
        